In [23]:
import hashlib
#HASH FUNCTION
def calculate_md5(input_string):
    md5_hash = hashlib.md5()
    md5_hash.update(input_string.encode('utf-8'))
    return md5_hash.hexdigest()

def make_plain_text(original_text):
    a = calculate_md5(original_text)
    return original_text+a

def build_key_map(key):
    sorted_key = sorted(list(key))
    return [sorted_key.index(k) for k in key]

#ENCRYPTION
def encrypt_transposition(plaintext, key):
    n = len(key)
    key_map = build_key_map(key)
    extra = len(plaintext) % n
    if extra != 0:
        plaintext += '$' * (n - extra)  
    rows = [plaintext[i:i+n] for i in range(0, len(plaintext), n)]
    # print(rows)
    ciphertext = []
    for col_index in range(n):
        actual_col = key_map.index(col_index)
        for row in rows:
            ciphertext.append(row[actual_col])
    return ''.join(ciphertext)

#DECRYPTION
def decrypt_transposition(ciphertext, key):
    n = len(key)
    key_map = build_key_map(key)
    num_rows = len(ciphertext) // n

    cols = [''] * n
    idx = 0
    for col_index in range(n):
        actual_col = key_map.index(col_index)
        cols[actual_col] = ciphertext[idx : idx + num_rows]
        idx += num_rows

    plaintext_chars = []
    for r in range(num_rows):
        for c in range(n):
            plaintext_chars.append(cols[c][r])
    plaintext = ''.join(plaintext_chars).rstrip('$')
    return plaintext

def pi_property(decrypted_text: str) -> bool:

  original_text = decrypted_text[:-32]
  hash_value = decrypted_text[-32:]

  return (hash_value == str(calculate_md5(original_text)))

In [24]:
key = input("Enter the key upto length 9 (Unique characters only): ")
def test_encrypt_decrypt(key):
    #read the plaintexts from the file
    with open('originaltext.txt','r') as f:
        plaintexts = f.readlines()
    plaintexts = [x.strip() for x in plaintexts]
    ciphertexts = []
    for plaintext in plaintexts:
        print("Original text: ", plaintext)
        plaintext = make_plain_text(plaintext)
        print("Plain_text: ", plaintext)
        encrypted_text = encrypt_transposition(plaintext, key)
        print("Ciphertext: ",encrypted_text)
        ciphertexts.append(encrypted_text)
        decrypted_text = decrypt_transposition(encrypted_text,key)
        print("Decryptedtext: ",decrypted_text)
        print("Matches original plaintext?: ", pi_property(decrypted_text))
        print()
    #write the cipher texts to the file
    with open('ciphertext.txt','w') as f:
        for ciphertext in ciphertexts:
            f.write(ciphertext+'\n')

test_encrypt_decrypt(key)


Original text:  helloalgorithm
Plain_text:  helloalgorithmbf166fa043b9fd1b4b3ed9271db67922
Ciphertext:  haifa949b2elt10fb26$lgh64d377$lom631e19$orbfbbdd2$
Decryptedtext:  helloalgorithmbf166fa043b9fd1b4b3ed9271db67922
Matches original plaintext?:  True

Original text:  securetransmission
Plain_text:  securetransmission23b2023154ac651c86b42282c0e2605d
Ciphertext:  sesib16682etmo255b26crin0414c0uas22ac205rns33c82ed
Decryptedtext:  securetransmission23b2023154ac651c86b42282c0e2605d
Matches original plaintext?:  True

Original text:  plaintextmessage
Plain_text:  plaintextmessage5d46f828bd89a39deb827ea4e31338c8
Ciphertext:  pteefd92e8les588d73caxsd29ee18ita48aba3$nmg6b3843$
Decryptedtext:  plaintextmessage5d46f828bd89a39deb827ea4e31338c8
Matches original plaintext?:  True

Original text:  cryptographyrules
Plain_text:  cryptographyrulesf9be7d1af5bb9e35908497a2abd96b2d
Ciphertext:  coheefe826rgys7534abyrrfdb59b2pau91b97ddtplba90a9$
Decryptedtext:  cryptographyrulesf9be7d1af5bb9e35908497a2ab

In [25]:
from itertools import permutations

def brute_force_attack(ciphertexts, alphabet=''.join(sorted(key)), max_key_length=9):
    for length in range(1, max_key_length + 1):
        for key_tuple in permutations(alphabet, length):
            key = ''.join(key_tuple)
            # print(key)
            if all(check_key_with_ciphertext(key, ct) for ct in ciphertexts):
                return key
    return None  

def check_key_with_ciphertext(key, ciphertext):
    decrypted_text = decrypt_transposition(ciphertext, key)
    return pi_property(decrypted_text)

def run_brute_force():

    #read the messages from the file
    messages = []
    with open("originaltext.txt") as f:
        messages = f.readlines()
    messages = [x.strip() for x in messages]


    ciphertexts = [encrypt_transposition(make_plain_text(msg), key) for msg in messages]
    discovered_key = brute_force_attack(ciphertexts)
    if discovered_key:
        print(f"Key found: {discovered_key}")
        for ct in ciphertexts:
            decrypted_text = decrypt_transposition(ct, discovered_key)
            print(f"Decrypted Text: {decrypted_text}")
    else:
        print("No valid key found.")

run_brute_force()


Key found: abcde
Decrypted Text: helloalgorithmbf166fa043b9fd1b4b3ed9271db67922
Decrypted Text: securetransmission23b2023154ac651c86b42282c0e2605d
Decrypted Text: plaintextmessage5d46f828bd89a39deb827ea4e31338c8
Decrypted Text: cryptographyrulesf9be7d1af5bb9e35908497a2abd96b2d
Decrypted Text: federatedlearninge9f950395073828bcdd1c508138ce9f5
